# Обучение с учителем в Scikit-learn.

### Задание 1

Импортируйте библиотеки `pandas` и `numpy`. Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки `sklearn`. Создайте датафреймы `X` и `y` из этих данных. Разбейте эти датафреймы на тренировочные (`X_train`, `y_train`) и тестовые (`X_test`, `y_test`) с помощью функции `train_test_split` так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент `random_state` должен быть равен 42. Создайте модель линейной регрессии под названием `lr` с помощью класса `LinearRegression` из модуля `sklearn.linear_model`. Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.
Вычислите `R2` полученных предказаний с помощью `r2_score` из модуля `sklearn.metrics`.


In [69]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [70]:
boston = load_boston()
data = boston["data"]
feature_names = boston["feature_names"]
target = boston["target"]

In [71]:
X = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=["price"])

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Линейной регрессия LinearRegression**

In [73]:
from sklearn.linear_model import LinearRegression

In [74]:
lr = LinearRegression()

In [75]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [76]:
y_pred = lr.predict(X_test)

In [77]:
r2_lr = r2_score(y_pred, y_test)
r2_lr

0.6693702691495595

### Задание 2

Создайте модель под названием `model` с помощью `RandomForestRegressor` из модуля `sklearn.ensemble`. Сделайте агрумент `n_estimators` равным 1000, `max_depth` должен быть равен 12 и `random_state` сделайте равным 42. Обучите модель на тренировочных данных аналогично тому, как вы обучали модель `LinearRegression`, но при этом в метод `fit` вместо датафрейма `y_train` поставьте `y_train.values[:, 0]`, чтобы получить из датафрейма одномерный массив `Numpy`, так как для класса `RandomForestRegressor` в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма. Сделайте предсказание на тестовых данных и посчитайте `R2`. Сравните с результатом из предыдущего задания. Напишите в комментариях к коду, какая модель в данном случае работает лучше.

**Линейной регрессия RandomForestRegressor**

In [78]:
from sklearn.ensemble import RandomForestRegressor

In [79]:
rfr = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

In [80]:
rfr.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [81]:
y_pred = rfr.predict(X_test)

In [82]:
r2_rfr = r2_score(y_pred, y_test.values[:, 0])
r2_rfr

0.8481499145965063

In [83]:
print('R2 для модели LinearRegression = {}\n'.format(r2_lr))
print('R2 для модели RandomForestRegressor = {}\n'.format(r2_rfr))

R2 для модели LinearRegression = 0.6693702691495595

R2 для модели RandomForestRegressor = 0.8481499145965063



Модель `RandomForestRegressor` оказалась более точной. Предположительно это связано с тем, что модель случайного леса не чувствительна к ненормализованным данным.

### * Задание 3

Вызовите документацию для класса `RandomForestRegressor`, найдите информацию об атрибуте `feature_importances_`. С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.

In [84]:
# ?RandomForestRegressor

In [85]:
feature_importances = rfr.feature_importances_
feature_importances

array([0.03211748, 0.00154999, 0.0070941 , 0.0011488 , 0.01436832,
       0.40270459, 0.01424477, 0.06403265, 0.00496762, 0.01169177,
       0.01808961, 0.0123114 , 0.41567892])

In [86]:
feature_importances_sum = feature_importances.sum()
feature_importances_sum

0.9999999999999999

In [87]:
df = pd.DataFrame({
    "feature": feature_names,
    "importance": feature_importances,
})
df

,feature,importance
0,CRIM,0.032117
1,ZN,0.001550
2,INDUS,0.007094
3,CHAS,0.001149
4,NOX,0.014368
5,RM,0.402705
6,AGE,0.014245
7,DIS,0.064033
8,RAD,0.004968
9,TAX,0.011692


In [88]:
df.nlargest(2, "importance")

,feature,importance
12,LSTAT,0.415679
5,RM,0.402705


Сумма значимости всех признаков равнв 1. Наиболее значимые признаки `LSTAT` и `RM`.

### *Задание 4

В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке `Matplotlib`, это датасет "Credit Card Fraud Detection". Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакциции по кредитной карте являются мошенническими. Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки), так что применение метрики `accuracy` не принесет пользы и не поможет выбрать лучшую модель. Мы будем вычислять `AUC`, то есть площадь под кривой `ROC`. 

Импортируйте из соответствующих модулей `RandomForestClassifier`, `GridSearchCV` и `train_test_split`. 

Загрузите датасет `creditcard.csv` и создайте датафрейм `df`.

С помощью метода `value_counts` с аргументом `normalize=True` убедитесь в том, что выборка несбалансирована. Используя метод `info`, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков. Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма: 

```
pd.options.display.max_columns = 100
```

Просмотрите первые 10 строк датафрейма `df`.

Создайте датафрейм `X` из датафрейма `df`, исключив столбец `Class`.

Создайте объект `Series` под названием `y` из столбца `Class`.

Разбейте `X` и `y` на тренировочный и тестовый наборы данных при помощи функции `train_test_split`, используя аргументы: 

```
test_size=0.3, random_state=100, stratify=y
```

У вас должны получиться объекты `X_train`, `X_test`, `y_train` и `y_test`.

Просмотрите информацию о их форме.

Для поиска по сетке параметров задайте такие параметры:

```
parameters = [{
    'n_estimators': [10, 15],
    'max_features': np.arange(3, 5),
    'max_depth': np.arange(4, 7)
}]
```

Создайте модель `GridSearchCV` со следующими аргументами:

```
estimator=RandomForestClassifier(random_state=100)
param_grid=parameters
scoring='roc_auc'
cv=3
```

Обучите модель на тренировочном наборе данных (может занять несколько минут).

Просмотрите параметры лучшей модели с помощью атрибута `best_params_`.

Предскажите вероятности классов с помощью полученнной модели и метода `predict_proba`.

Из полученного результата (массив `Numpy`) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив `y_pred_proba`. Из модуля `sklearn.metrics` импортируйте метрику `roc_auc_score`.

Вычислите `AUC` на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументовмассивы `y_test` и `y_pred_proba`.

### *Дополнительные задания:

##### 1. Загрузите датасет Wine из встроенных датасетов sklearn.datasets с помощью функции load_wine в переменную data.

##### 2. Полученный датасет не является датафреймом. Это структура данных, имеющая ключи аналогично словарю. Просмотрите тип данных этой структуры данных и создайте список data_keys, содержащий ее ключи.

##### 3. Просмотрите данные, описание и названия признаков в датасете. Описание нужно вывести в виде привычного, аккуратно оформленного текста, без обозначений переноса строки, но с самими переносами и т.д.

##### 4. Сколько классов содержит целевая переменная датасета? Выведите названия классов.

##### 5. На основе данных датасета (они содержатся в двумерном массиве Numpy) и названий признаков создайте датафрейм под названием X.

##### 6. Выясните размер датафрейма X и установите, имеются ли в нем пропущенные значения.

##### 7. Добавьте в датафрейм поле с классами вин в виде чисел, имеющих тип данных numpy.int64. Название поля - 'target'.

##### 8. Постройте матрицу корреляций для всех полей X. Дайте полученному датафрейму название X_corr.

##### 9. Создайте список high_corr из признаков, корреляция которых с полем target по абсолютному значению превышает 0.5 (причем, само поле target не должно входить в этот список).

##### 10. Удалите из датафрейма X поле с целевой переменной. Для всех признаков, названия которых содержатся в списке high_corr, вычислите квадрат их значений и добавьте в датафрейм X соответствующие поля с суффиксом '_2', добавленного к первоначальному названию признака. Итоговый датафрейм должен содержать все поля, которые, были в нем изначально, а также поля с признаками из списка high_corr, возведенными в квадрат. Выведите описание полей датафрейма X с помощью метода describe.
